In [4]:
from bs4 import BeautifulSoup
from IPython.display import clear_output
from datetime import date, datetime, timedelta
import requests, urllib.request, urllib.parse, csv, re, time, os, numpy as np, pandas as pd

In [5]:
def daum_url(start, end, query, press_name) :
    press_id_lists = {'경향신문':'16bfGN9mQcFhOx4F5l',
                  '한겨레':'16CIYSC5zGTVsMKcxM',
                  '조선일보':'16EeZKAuilXKH5dzIt',
                  '중앙일보':'16nfco03BTHhdjCcTS',
                  '동아일보':'16Et2OLVVtHab8gcjE',
                  '국민일보':'16NwX_ox536G_zyJUF',
                  '한국일보':'16hsvX4VEJdcIZzt_z',
                  '한국경제':'16qCuwnoTf8fLmrhD1',
                  '매일경제':'16jCK_TdtzwnmXfznB'}
    pre_url = "https://search.daum.net/search?w=news&sort=recency&q=%s&cluster=n&s=NS&a=STCF&dc=STC&pg=1&r=1&p=%d&rc=1&at=more&sd=%d000000&ed=%d235959&period=u&cp=%s&cpname=%s"
    url = pre_url %(urllib.parse.quote(query),
                    1,
                    start,
                    end,
                    press_id_lists[str(press_name)],
                    urllib.parse.quote(press_name))
    return(url)

In [6]:
def daum_url_pagenum(start, end, query, press_name, page_num) :
    press_id_lists = {'경향신문':'16bfGN9mQcFhOx4F5l',
                  '한겨레':'16CIYSC5zGTVsMKcxM',
                  '조선일보':'16EeZKAuilXKH5dzIt',
                  '중앙일보':'16nfco03BTHhdjCcTS',
                  '동아일보':'16Et2OLVVtHab8gcjE',
                  '국민일보':'16NwX_ox536G_zyJUF',
                  '한국일보':'16hsvX4VEJdcIZzt_z',
                  '한국경제':'16qCuwnoTf8fLmrhD1',
                  '매일경제':'16jCK_TdtzwnmXfznB'}
    pre_url = "https://search.daum.net/search?w=news&sort=recency&q=%s&cluster=n&s=NS&a=STCF&dc=STC&pg=1&r=1&p=%d&rc=1&at=more&sd=%d000000&ed=%d235959&period=u&cp=%s&cpname=%s"
    url = pre_url %(urllib.parse.quote(query),
                    page_num,
                    start,
                    end,
                    press_id_lists[str(press_name)],
                    urllib.parse.quote(press_name))
    return(url)

In [7]:
def daum_news_scraper(start, end, query, press_name) :
    news_links = []
    date_list = [datetime.strptime(str(end),'%Y%m%d') - timedelta(days=x) 
             for x in range(0, (datetime.strptime(str(end),'%Y%m%d')-datetime.strptime(str(start),'%Y%m%d')).days+1)]
    session = requests.Session()
    header = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5)\AppleWebKit 537.36 (KHTML, like Gecko) Chrome",
              "Accept":"text/html,application/xhtml+xml,application/xml;\q=0.9,imgwebp,*/*;q=0.8"}
    url = daum_url(start, end, query, press_name)
    req = session.get(url, headers=header)
    soup = BeautifulSoup(req.text, 'html.parser')
    total_num_article = int(''.join(re.findall('\d+',re.search(r'/ (.*?)건',soup.find('span', attrs={'class':'txt_info'}).text).group(1))))
    
    if total_num_article <= 800 :
        for each in range(1,(total_num_article//10)+2) :
            time.sleep(.25)
            new_soup = BeautifulSoup(session.get(daum_url_pagenum(start, end, query, press_name, each), headers=header).text, 'html.parser')
            for each_link in new_soup.find_all('div', attrs={'class':'wrap_tit mg_tit'}) :
                news_links.append(each_link.a['href'])
                clear_output(wait=True)
                print(len(news_links),'/',total_num_article)                
        
    else :
        for each_date in date_list :
            time.sleep(.25)
            new_url = daum_url(int(each_date.strftime('%Y%m%d')), int(each_date.strftime('%Y%m%d')), query, press_name)
            new_req = session.get(new_url, headers=header)
            new_soup = BeautifulSoup(new_req.text, 'html.parser')
            num_article = int(''.join(re.findall('\d+',re.search(r'/ (.*?)건',new_soup.find('span', attrs={'class':'txt_info'}).text).group(1))))

            if num_article == 0 :
                pass
            else :
                for each_page in range(1,(num_article//10)+2) :
                    time.sleep(.25)
                    new_soup2 = BeautifulSoup(session.get(
                        daum_url_pagenum(int(each_date.strftime('%Y%m%d')), int(each_date.strftime('%Y%m%d')), query, press_name, each_page), headers=header).text, 'html.parser')
                    for each_link in new_soup2.find_all('div', attrs={'class':'wrap_tit mg_tit'}) :
                        news_links.append(each_link.a['href'])
                        clear_output(wait=True)
                        print(len(news_links),'/',total_num_article)
    print(len(news_links),'/',total_num_article, 'COMPLETE!')
    pd.DataFrame(news_links).to_csv(str(start)+'_'+str(end)+'_'+query+'_'+press_name+'.csv')
    return(news_links)